	# Author: Alexander Staub
	## Last changed: 2023.03.21
	## Purpose: Accessing the Spotify API


# Done to dos:
* rate limits: 10/second without rate limit extension
* changed the .append to .concat in the loop to avoid warnings 
* added the audio feature elements from spotify - all of them according to gpt-4


# Next up to dos - 23.03.28

* with the first random sample, a number of songs were not matched. Think of a way to get all the songs that weren't matched into a seperate dataframe


In [26]:
#library loading

import os
import spotipy # lightweight Python library for the Spotify Web API
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials # to access authorised Spotify data
import json # to read json files
import time # to time the code
import requests # to make http requests

In [10]:
# path specification - using os

# Get the current working directory
current_dir = os.getcwd()

# Navigate up two levels to the root directory
root_dir = os.path.abspath(os.path.join(current_dir, '..', '..'))

print(root_dir)


c:\Users\User\Documents\R_Work\research\music_data


In [4]:
# get the client id and client secret for my account

# access the spotify credentials from the json file and store it as variable "f"
with open('spotify_credentials.json') as f:

    # transform the json file into a dictionary
    data = json.load(f)
    client_id = data['SPOTIPY_CLIENT_ID']
    client_secret = data['SPOTIPY_CLIENT_SECRET']

if not client_id or not client_secret:
    raise ValueError("Spotify API credentials not found in environment variables")

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

In [13]:
# load in the random sample w isrcs into a dataframe

# Create the path to the CSV file
csv_file_path = os.path.join(root_dir, 'data', 'interim_data', 'random_sample_w_iscrcs_mb_v5_export.csv')

In [14]:
#detect the encoding of the csv

import chardet    

rawdata = open(csv_file_path, "rb").read()
result = chardet.detect(rawdata)
charenc = result['encoding']

In [16]:
# Now read the CSV into a DataFrame
sample_songs_w_isrcs = pd.read_csv(csv_file_path, encoding=charenc)

#remove the column named "unnamed" from the sample_songs_w_isrcs dataframe
sample_songs_w_isrcs = sample_songs_w_isrcs.loc[:, ~sample_songs_w_isrcs.columns.str.contains('^Unnamed')]


Currently, the loaded dataframe includes variables of the type "object" - which means that they are some generic form of data type.
Converting all but date to string will make it easier to access spotify API

In [19]:
#loop through all variables and convert them to strings - except the date column
for col in sample_songs_w_isrcs.columns:
    if col != 'date':
        sample_songs_w_isrcs[col] = sample_songs_w_isrcs[col].astype(str)



The code below is meant to:
- check the ISRC codes in the musicbrainz dataframe and collect the name of thesong, artist and acoustic charactersitics from the Spotify Api
- respect the rate limits of 10 requests per second
- save the data periodically in case of some error
- collect data about unmatched isrcs in a seperate file for later review
- not restart from 0 in case the operation fails, but check what was already collected

In [21]:
# create the save path
save_path = os.path.join(root_dir, "data", "interim_data", "spotify_isrc_sample")

print(save_path)

c:\Users\User\Documents\R_Work\research\music_data\data\interim_data\spotify_isrc_sample


In [29]:
#try whether the the dataframe already exists or whether it needs to be created

try:
    df_spotify = pd.read_csv(os.path.join(save_path, "partial_spotify_data.csv"))
except FileNotFoundError:
    df_spotify = pd.DataFrame()

try:
    with open(os.path.join(save_path, "unmatched_isrcs.json"), "r") as f:
        unmatched_isrcs = json.load(f)
except FileNotFoundError:
    unmatched_isrcs = []

# List to hold dictionaries to append to df_spotify
data_to_append = []  


In [30]:
# Track where you are in the loop in case you have to restart
start_point = 0
try:
    # Check if a start_point file exists (this assumes the file holds an integer)
    with open(os.path.join(save_path, "start_point.txt"), "r") as f:
        start_point = int(f.read())
except FileNotFoundError:
    pass

In [ ]:
# Loop through your Musicbrainz dataframe
for i, row in sample_songs_w_isrcs.iloc[start_point:].iterrows():
    
    isrc = row['ISRC']
    
    try:
        # Search for track by ISRC
        results = sp.search(q=f'isrc:{isrc}', type='track')
        
        # Check if the search returned any tracks
        if results['tracks']['items']:
            track = results['tracks']['items'][0]
            
            # Fetch audio features using Spotify ID
            audio_features = sp.audio_features(track["id"])[0]

            # Extract and append information to df_spotify
            if audio_features is not None:
                track_info = {
                    "ISRC": isrc,
                    "name": track["name"],
                    "artist": track["artists"][0]["name"],
                    **audio_features
                }
                data_to_append.append(track_info)
            else:
                print(f"Audio features missing for ISRC: {isrc}")
                unmatched_isrcs.append((i, isrc))
        else:
            print(f"Couldn't match ISRC: {isrc}")
            unmatched_isrcs.append((i, isrc))

    except requests.exceptions.HTTPError as err:
        if err.response.status_code == 429:  # HTTP Status Code for Too Many Requests
            time.sleep(1)  # Pause for 1 second and then retry
            continue
        else:
            print(f"HTTP Error for ISRC: {isrc}, Error: {err}")
            unmatched_isrcs.append((i, isrc))
        
    # Manage rate limits (Spotify API allows 10 requests per second)
    time.sleep(0.2)
    
    # Save start_point
    with open(os.path.join(save_path, "start_point.txt"), "w") as f:
        f.write(str(i))
    
    # You may also want to save your results periodically to a CSV or JSON
    if i % 100 == 0:
        df_spotify.to_csv(os.path.join(save_path, "partial_spotify_data.csv"), index=False)
        with open(os.path.join(save_path, "unmatched_isrcs.json"), "w") as f:
            json.dump(unmatched_isrcs, f)


#Use pandas.concat to append all rows at once
if data_to_append:
    df_spotify = pd.concat([df_spotify, pd.DataFrame(data_to_append)], ignore_index=True)


# Save your results
df_spotify.to_csv(os.path.join(save_path, "complete_spotify_data.csv"), index=False)
with open(os.path.join(save_path, "unmatched_isrcs.json"), "w") as f:
    json.dump(unmatched_isrcs, f)

# First random sample of songs - without ISRC
To do:
- move this into a new Jupyter notebook

In [1]:
import os
import spotipy # lightweight Python library for the Spotify Web API
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials # to access authorised Spotify data

In [2]:
# package for accessing the local environment variables
import os

# get the client id and client secret from the environment variables
client_id = os.environ.get('SPOTIPY_CLIENT_ID')
client_secret = os.environ.get('SPOTIPY_CLIENT_SECRET')

if not client_id or not client_secret:
    raise ValueError("Spotify API credentials not found in environment variables")

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

In [3]:
#required package

from pandas import read_csv

# loading in the csv file with the songs from billboard and musicbrainz

df_HH_mb_matched = read_csv('C:/R work/Research/music_data/data/interim_data/df_hh_and_mb_leftjoin_fuzzy.csv')

In [5]:
# draw a random observation from the dataframe, retaining only the first two columns 
random_100_df = df_HH_mb_matched.sample(100)[['Artist', 'Track']]



In [6]:
#create the song infor retreival function

# added a bit to the function to handle errors and retries
import time

#here the max retries is 3
def get_song_data(song_name, artist_name, max_retries=3):
    query = f'track:{song_name} artist:{artist_name}'
    retries = 0
    
    #while loop to retry the query if it fails n times (as defined by max_retries)
    while retries <= max_retries:
        try:
            results = sp.search(q=query, type='track', limit=1)
            
            if results['tracks']['items']:
                track = results['tracks']['items'][0]

               # Get the track's ID
                track_id = track['id']
                
                # Get the audio features for the track
                audio_features = sp.audio_features([track_id])[0]

                song_data = {
                    'song_name': track['name'],
                    'artist_name': track['artists'][0]['name'],
                    'album_name_spotify': track['album']['name'],
                    'release_date_spotify': track['album']['release_date'],
                    'popularity': track['popularity'],
                    'explicit': track['explicit'],
                    'duration_ms': track['duration_ms'],
                    # 'preview_url': track['preview_url'],
                    'acousticness': audio_features['acousticness'],
                    'danceability': audio_features['danceability'],
                    'energy': audio_features['energy'],
                    'instrumentalness': audio_features['instrumentalness'],
                    'liveness': audio_features['liveness'],
                    'loudness': audio_features['loudness'],
                    'speechiness': audio_features['speechiness'],
                    'valence': audio_features['valence'],
                    'tempo': audio_features['tempo'],
                    'key': audio_features['key'],
                    'mode': audio_features['mode'], # mode is either major or minor
                    'time_signature': audio_features['time_signature']
                }
                return song_data
            else:
                return None
        
        except Exception as e:
            print(f"Error: {e}")
            retries += 1
            sleep_time = 2 ** retries
            print(f"Retrying ({retries}/{max_retries}) after {sleep_time} seconds")
            time.sleep(sleep_time)
    
    return None



In [11]:
# create an empty dataframe
results_df = pd.DataFrame(columns=["Track", "Artist", "song_name", "artist_name", "album_name_spotify", "release_date_spotify",
                                    "popularity", "explicit", "duration_ms", "acousticness", "danceability","energy","instrumentalness",
                                    "liveness", "loudness", "speechiness", "valence", "tempo", "key", "mode", "time_signature"])

# loop over the random sample dataframe

rest_period = 0.2  # Time in seconds between requests

for index, row in random_100_df.iterrows():
    song_name = row["Track"]
    artist_name = row["Artist"]

    song_data = get_song_data(song_name, artist_name)
    if song_data:
        song_data["Track"] = song_name
        song_data["Artist"] = artist_name

        # Convert song_data dictionary to a DataFrame
        song_data_df = pd.DataFrame([song_data])

        # changed results.append to results_df.concat as per the errors in the previous code
        
        results_df = pd.concat([results_df, song_data_df], ignore_index=True)
    else:
        print(f"Song not found: {song_name} by {artist_name}")
        
    time.sleep(rest_period)  # Pause execution for the specified rest period    



C:\Users\astaub\AppData\Local\Temp\ipykernel_16592\1066911844.py:24: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  results_df = pd.concat([results_df, song_data_df], ignore_index=True)


Song not found: Karma by Lloyd Banks Featuring Avant


C:\Users\astaub\AppData\Local\Temp\ipykernel_16592\1066911844.py:24: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  results_df = pd.concat([results_df, song_data_df], ignore_index=True)
C:\Users\astaub\AppData\Local\Temp\ipykernel_16592\1066911844.py:24: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  results_df = pd.concat([results_df, song_data_df], ignore_index=True)
C:\Users\astaub\AppData\Local\Temp\ipykernel_16592\1066911844.py:24: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  results_df = pd.concat([results_df, song_data_df], ignore_index=True)
C:\Users\astaub\AppData\Local\Temp\ipykernel_16592\1066911

Song not found: Area Codes by Ludacris Featuring Nate Dogg


C:\Users\astaub\AppData\Local\Temp\ipykernel_16592\1066911844.py:24: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  results_df = pd.concat([results_df, song_data_df], ignore_index=True)
C:\Users\astaub\AppData\Local\Temp\ipykernel_16592\1066911844.py:24: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  results_df = pd.concat([results_df, song_data_df], ignore_index=True)
C:\Users\astaub\AppData\Local\Temp\ipykernel_16592\1066911844.py:24: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  results_df = pd.concat([results_df, song_data_df], ignore_index=True)
C:\Users\astaub\AppData\Local\Temp\ipykernel_16592\1066911

...

Song not found: Oochie Wally by QB Finest Featuring Nas & Bravehearts


C:\Users\astaub\AppData\Local\Temp\ipykernel_16592\1066911844.py:24: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  results_df = pd.concat([results_df, song_data_df], ignore_index=True)
C:\Users\astaub\AppData\Local\Temp\ipykernel_16592\1066911844.py:24: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  results_df = pd.concat([results_df, song_data_df], ignore_index=True)
C:\Users\astaub\AppData\Local\Temp\ipykernel_16592\1066911844.py:24: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  results_df = pd.concat([results_df, song_data_df], ignore_index=True)
C:\Users\astaub\AppData\Local\Temp\ipykernel_16592\1066911

Song not found: I'm Your Angel by R. Kelly & Celine Dion


C:\Users\astaub\AppData\Local\Temp\ipykernel_16592\1066911844.py:24: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  results_df = pd.concat([results_df, song_data_df], ignore_index=True)


Song not found: Work It by Missy "Misdemeanor" Elliott
